In [71]:
import pandas as pd

df = pd.read_json('aspen.json')

In [72]:
import os
import sys
import numpy as np


#pre-processing of text
import string
import re


from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [73]:
# Convert the lists in 'hotel' column to strings
df['hotel'] = df['hotel'].apply(lambda x: x[0])  # Convert list to string
df['content'] = df['content'].apply(lambda x: x[0]) 
# Combine 'hotel' and 'clear_text' into a single column
df['combined'] = df['hotel'] + ":\n"  + df['content']

# Drop the original columns if needed
#df.drop(['hotel', 'content'], axis=1, inplace=True)

In [74]:
df

,stars,hotel,title,content,combined
0,5,Limelight Hotel,[Awesome visit ],Went on a girls trip this past weekend. We had...,Limelight Hotel:\nWent on a girls trip this pa...
1,5,Limelight Hotel,[Super hotel and Super Staff],We were very lucky to win 4 nights accomodatio...,Limelight Hotel:\nWe were very lucky to win 4 ...
2,5,Chateau Roaring Fork,[Wait until the last minute],Wait until the last minute and get a lodging/l...,Chateau Roaring Fork:\nWait until the last min...
3,5,Limelight Hotel,[Great Hotel. Nice place to stay],Great hotel. Beautiful. Great well decorated b...,Limelight Hotel:\nGreat hotel. Beautiful. Grea...
4,3,Aspen Mountain Lodge,[Quaint and cozy lodge],"Great value, decent location. I'd highly recom...","Aspen Mountain Lodge:\nGreat value, decent loc..."
...,...,...,...,...,...
2001,5,Chateau Blanc,[Great stay],Spent a couple of nights in Aspen on a girls g...,Chateau Blanc:\nSpent a couple of nights in As...
2002,4,Chateau Blanc,[Excellent cost/benefit],We stayed in a two bedrooms/bathrooms apartmen...,Chateau Blanc:\nWe stayed in a two bedrooms/ba...
2003,5,Chateau Blanc,[Great WInter Vaca],A wonderful place to stay for our family vacat...,Chateau Blanc:\nA wonderful place to stay for ...
2004,5,Chateau Blanc,[Chateau Blanc for a week],The lodge is few blocks away from the main dow...,Chateau Blanc:\nThe lodge is few blocks away f...


In [75]:
df['combined']= df['combined'].str.lower()


In [76]:
df['combined']

0       limelight hotel:\nwent on a girls trip this pa...
1       limelight hotel:\nwe were very lucky to win 4 ...
2       chateau roaring fork:\nwait until the last min...
3       limelight hotel:\ngreat hotel. beautiful. grea...
4       aspen mountain lodge:\ngreat value, decent loc...
                              ...                        
2001    chateau blanc:\nspent a couple of nights in as...
2002    chateau blanc:\nwe stayed in a two bedrooms/ba...
2003    chateau blanc:\na wonderful place to stay for ...
2004    chateau blanc:\nthe lodge is few blocks away f...
2005    shadow mountain lodge:\ngreat location, nice r...
Name: combined, Length: 2006, dtype: object

In [77]:
# Save 'combined' column to a text file
df['combined'].to_csv('combined_text.txt', index=False, header=False, sep='\t')  # Change sep='\t' to sep=' ' if needed

print("Combined text saved to 'combined_text.txt'")

Combined text saved to 'combined_text.txt'


In [79]:
import os
import time
import numpy as np
import tensorflow as tf

2024-04-16 15:22:59.848714: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [81]:
text = open("combined_text.txt", 'rb').read().decode(encoding='utf-8')
print(f"Lenght of text: {len(text)} characters")


Lenght of text: 948809 characters


In [82]:
print(text[:250])

"limelight hotel:
went on a girls trip this past weekend. we had a wonderful time and the hotel was amazing. everyone from the front desk to the shuttle drivers to the bartenders and kitchen staff were fantastic. the breakfast and warm cookies were a


In [83]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

77 unique characters


### Process the text

In [87]:
example_text = ['stefan', 'nafets']

chars = tf.strings.unicode_split(example_text, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b's', b't', b'e', b'f', b'a', b'n'],
 [b'n', b'a', b'f', b'e', b't', b's']]>

In [88]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None)

In [89]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[53, 54, 39, 40, 35, 48],
 [48, 35, 40, 39, 54, 53]]>

In [90]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None)

In [91]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b's', b't', b'e', b'f', b'a', b'n'],
 [b'n', b'a', b'f', b'e', b't', b's']]>

In [92]:
tf.strings.reduce_join(chars, axis=1).numpy()


array([b'stefan', b'nafets'], dtype=object)

In [94]:
# lets create a function that we can call later
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=0)

### The prediciton task

In [95]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(948809,), dtype=int64, numpy=array([ 4, 46, 43, ..., 16,  4,  1])>

In [97]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [98]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('UTF-8'))

"
l
i
m
e
l
i
g
h
t


In [99]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

In [100]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'"' b'l' b'i' b'm' b'e' b'l' b'i' b'g' b'h' b't' b' ' b'h' b'o' b't'
 b'e' b'l' b':' b'\n' b'w' b'e' b'n' b't' b' ' b'o' b'n' b' ' b'a' b' '
 b'g' b'i' b'r' b'l' b's' b' ' b't' b'r' b'i' b'p' b' ' b't' b'h' b'i'
 b's' b' ' b'p' b'a' b's' b't' b' ' b'w' b'e' b'e' b'k' b'e' b'n' b'd'
 b'.' b' ' b'w' b'e' b' ' b'h' b'a' b'd' b' ' b'a' b' ' b'w' b'o' b'n'
 b'd' b'e' b'r' b'f' b'u' b'l' b' ' b't' b'i' b'm' b'e' b' ' b'a' b'n'
 b'd' b' ' b't' b'h' b'e' b' ' b'h' b'o' b't' b'e' b'l' b' ' b'w' b'a'
 b's' b' ' b'a'], shape=(101,), dtype=string)


In [101]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'"limelight hotel:\nwent on a girls trip this past weekend. we had a wonderful time and the hotel was a'
b'mazing. everyone from the front desk to the shuttle drivers to the bartenders and kitchen staff were '
b'fantastic. the breakfast and warm cookies were a delicious touch also. we will definitely be back nex'
b't year for more fun!"\n"limelight hotel:\nwe were very lucky to win 4 nights accomodation at the limeli'
b'ght hotel and we stayed an extra 4 nights to make a full week of skiing.  it was our 7th trip to aspe'


In [102]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [103]:
split_input_target(list('Tensorflow'))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [104]:
dataset = sequences.map(split_input_target)

In [106]:
for input_example, target_example in dataset.take(1):
  print('Input  :', text_from_ids(input_example).numpy())
  print('Target :', text_from_ids(target_example).numpy())

Input  : b'"limelight hotel:\nwent on a girls trip this past weekend. we had a wonderful time and the hotel was '
Target : b'limelight hotel:\nwent on a girls trip this past weekend. we had a wonderful time and the hotel was a'


In [107]:
# Batch size
BATCH_SIZE = 63

"""Buffer size to shuffle the dataset
(TF data is designed to work wiht possobly infinite sequences,
so it dosen't attempt to shuffle the entire sequence in memory. Instadfe
it maintaines a buffer is which it shuffles elements)"""
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE) \
    .batch(BATCH_SIZE, drop_remainder=True) \
    .prefetch(tf.data.experimental.AUTOTUNE)
)
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(63, 100), dtype=tf.int64, name=None), TensorSpec(shape=(63, 100), dtype=tf.int64, name=None))>

### Build The Model

In [188]:
# Length of the vocabulart in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [189]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        """since we are training a text generation model,
        we use the previous state, in training. If there is no state,
        then we initialize the state """
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x


In [195]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, lstm_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(
            lstm_units,
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        """由于我们正在训练一个文本生成模型，
        我们在训练中使用前一个状态。如果没有状态，
        则初始化状态 """
        if states is None:
            # LSTM层会管理自己的初始状态
            x, state_h, state_c = self.lstm(x, training=training)
        else:
            # 将初始状态传递给LSTM层
            x, state_h, state_c = self.lstm(x, initial_state=states, training=training)
        
        # 将LSTM层的输出通过密集层
        x = self.dense(x, training=training)

        if return_state:
            return x, [state_h, state_c]
        else:
            return x

In [194]:
# 1. Instantiate the model
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    lstm_units=rnn_units)

# 2. Prepare input data (example data)
input_data = [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]  # Example numerical sequences

# 3. Call the model to obtain predictions
predictions, _ = model(input_data)

# 4. Process the predictions
print("Predictions shape:", predictions.shape)


AttributeError: Exception encountered when calling layer 'embedding_13' (type Embedding).

'list' object has no attribute 'dtype'

Call arguments received by layer 'embedding_13' (type Embedding):
  • inputs=[['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)'], ['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)']]

In [190]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size = len(ids_from_chars.get_vocabulary()),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units)

In [191]:
# Process the dataset
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)"
    )

(63, 100, 78) # (batch_size, sequence_length, vocab_size)


In [196]:
model.summary()

Model: "my_model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    multiple                  19968     
                                                                 
 lstm_11 (LSTM)              multiple                  0 (unused)
                                                                 
 dense_13 (Dense)            multiple                  0 (unused)
                                                                 
Total params: 19968 (78.00 KB)
Trainable params: 19968 (78.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [127]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0],
    num_samples = 1
)

sampled_indices = tf.squeeze(sampled_indices, axis=1).numpy()

In [128]:
sampled_indices

array([58, 48, 71,  5, 47, 47, 44, 55, 52, 30, 56, 18, 31, 14, 28, 31, 51,
       32, 34, 16, 64, 61, 53, 21, 21, 70, 13,  2, 38, 47, 27,  0, 66, 13,
        7, 37, 60, 65, 30, 76, 48, 76, 27, 77, 74, 43, 39, 53,  9, 33, 37,
       68,  4, 61, 65, 63, 65, 50, 52, 69, 17, 32, 14,  5, 46,  8, 28, 45,
       11, 52, 57, 28, 74, 39, 51, 70, 55, 15, 70, 76, 61, 65,  9, 38, 71,
       70, 61, 37,  0, 18, 77, 17, 16, 19, 42, 47, 12, 74, 49, 58])

In [129]:
print('Input:', text_from_ids(input_example_batch[0]).numpy(), sep='\n')
print('Next Char Predictions:', text_from_ids(sampled_indices).numpy(), sep='\n')

Input:
b'ain chalet aspen:\nhotel location is great - you can walk to restaurants and major attractions."\n"mou'
Next Char Predictions:
b'xn\xe2\x80\x94#mmjur<v0=,:=q>\\.\xc2\xb0{s33\xe2\x80\x93+ dm9[UNK]\xc3\xa2+%cz\xc2\xb4<\xe2\x80\xa6n\xe2\x80\xa69\xef\xb8\x8f\xe2\x80\x9cies\'?c\xc3\xa9"{\xc2\xb4~\xc2\xb4pr\xc3\xbc/>,#l&:k)rw:\xe2\x80\x9ceq\xe2\x80\x93u-\xe2\x80\x93\xe2\x80\xa6{\xc2\xb4\'d\xe2\x80\x94\xe2\x80\x93{c[UNK]0\xef\xb8\x8f/.1hm*\xe2\x80\x9cox'


In [130]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [131]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    'Prediciton shape: ',
    example_batch_predictions.shape,
    '# (batch_size, sequence_length, vocab_size',
    sep='\n'
)
print('New loss:        ', example_batch_mean_loss)

Prediciton shape: 
(63, 100, 78)
# (batch_size, sequence_length, vocab_size
New loss:         tf.Tensor(4.3574915, shape=(), dtype=float32)


In [132]:
tf.exp(example_batch_mean_loss).numpy()

78.06107

In [133]:
model.compile(optimizer='adam', loss=loss)

In [134]:
# Path where tha checkpoints will be saved
checkpoint_path = './training_checkpoints'
# Name of the checkpoint file
checkpoint_prefix = os.path.join(checkpoint_path, "ckpt_(epoch)")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix, save_weight_only=True)

In [135]:
%%time
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
149/149 [==============================] - ETA: 0s - loss: 2.6708INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_(epoch)/assets


INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_(epoch)/assets


149/149 [==============================] - 362s 2s/step - loss: 2.6708
Epoch 2/10
149/149 [==============================] - ETA: 0s - loss: 1.8851INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_(epoch)/assets


INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_(epoch)/assets


149/149 [==============================] - 424s 3s/step - loss: 1.8851
Epoch 3/10
149/149 [==============================] - ETA: 0s - loss: 1.4637INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_(epoch)/assets


...

149/149 [==============================] - 293s 2s/step - loss: 1.0100
Epoch 9/10
149/149 [==============================] - ETA: 0s - loss: 0.9692INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_(epoch)/assets


INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_(epoch)/assets


149/149 [==============================] - 293s 2s/step - loss: 0.9692
Epoch 10/10
149/149 [==============================] - ETA: 0s - loss: 0.9304INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_(epoch)/assets


INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_(epoch)/assets


149/149 [==============================] - 292s 2s/step - loss: 0.9304
CPU times: user 2h 32min 36s, sys: 16min 15s, total: 2h 48min 52s
Wall time: 53min 30s


In [137]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
                # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [138]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
     

In [155]:
start = time.time()
states = None
next_char = tf.constant(['Chateau Roaring Fork:'])
result = [next_char]

for n in range(200):
    next_char, states = one_step_model.generate_one_step(
        next_char, states = states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), "\n\n" + "_" * 80)
print("\nRun time:", end - start)


Chateau Roaring Fork:
other outside, clean mountain aspen alpss was wonderful...a we will  bring you terry who i spend 3 night for the morning at their many marsel, beds, beds, and gave us right across the street was olde 

________________________________________________________________________________

Run time: 0.35026097297668457


In [152]:
start = time.time()
states = None
next_char = tf.constant(['limelight hotel','limelight hotel' ])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states = states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + "_" * 80)
print("\nRun time", end - start)

tf.Tensor(
[b'limelight hotel is warm brea...the provide suite also, night 4 add end large but it was not on the best hotels a great place to stay. the weekend could comy site four was great for our weekend. from the wonderful places; this well maintenance staff were very good.  staff was outstanding, all night, where more than oded, laundry, i spoky at aspen next trip to aspen crowds... down impercaben lonwe afternoon on the heart of aspen and excredible biking.  the new cunter queent parking based on day several times."\n"snow queen lodge:\ni was amazing and extra tjiditions were always made over the tv.  he was beine - and small while sleeping in the one bedroom condo (the bathroom was clean and had been updated to stay and was throughout the lodge the room. the breakfast and costs resorts and both were so clean. if i also read the bellman downstairs.cerence and limelight hotel:\na better than clean come layoners are indip with plenty on tho bedrooms above.  it was an insult i could

### Export the generator

In [136]:
# save the one step model
tf.saved_model.save(one_step_model, 'one_step')

NameError: name 'one_step_model' is not defined